In [2]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.7/407.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/

In [3]:
import os

#/* get API Key */
from google.colab import userdata
OPENAI_API= userdata.get('OPENAI_API')
LANGCHAIN_API_KEY= userdata.get('LANGCHAIN_API_KEY')
print(f"OpenAI API: {OPENAI_API[:4]}********")
print(f"LangChain API Key: {LANGCHAIN_API_KEY[:4]}********")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_ENDPOINT"]="https://api.smith.langchain.com"
os.environ["OPENAI_API_KEY"] = OPENAI_API  # This line is changed
os.environ["LANGCHAIN_PROJECT"]="tech4impact_1010"
os.environ["LANGCHAIN_API_KEY"]= LANGCHAIN_API_KEY

OpenAI API: sk-p********
LangChain API Key: lsv2********


LangSmith Setup (LLM Logging)


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8645a828-79b9-4bb0-beef-f16e6e292c00-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Starts from now

In [21]:
import os
import json
import getpass

# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [10]:
cd /content/drive/MyDrive/24fall/tenten/tenten-ai

/content/drive/MyDrive/24fall/tenten/tenten-ai


In [45]:
with open('transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)
utterances = data['utterances']
documents = []
for utterance in utterances:
    content = utterance['msg']
    metadata = {
        'start_at': utterance['start_at'],
        'duration': utterance['duration'],
        'speaker': utterance['spk'],
        'speaker_type': utterance['spk_type']
    }
    documents.append(Document(page_content=content, metadata=metadata))

documents = sorted(documents, key=lambda doc: doc.metadata['start_at'])


In [12]:
combined_content = '\n'.join([doc.page_content for doc in documents])
combined_document = Document(page_content=combined_content)

# 1. Split the documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=['\n', '.', '!', '?', ',', ' '],
    add_start_index=True
)
docs = text_splitter.split_documents(documents)

# 2. Embed w/ chromadb
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(docs, embeddings)


<ipython-input-12-04819deae8c8>:16: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


In [13]:
# 3. Set up the retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [40]:

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
     아래는 상담 세션의 전체 스크립트입니다. 이 스크립트를 시간의 흐름에 따라 **주제별**로 요약해 주세요.

    다음 조건을 따라 주세요:
    1. 주제는 가능하면 **질문 단위**로 묶어 주세요.
    2. 주제 중 약사의 조언은 다음 형식으로 구분해 주세요:
       - **[바꾸세요]**: 행동이나 습관을 변경해야 하는 경우
       - **[이렇게 하세요]**: 올바른 방법이나 권장 사항을 안내할 때

    예시:
    - Q. 당뇨 약을 일주일 간 끊어도 될까요?
    - Q. 오메가3 영양제는 효과가 있나요?
    - [바꾸세요] 항생제는 중단하지 마세요.
    - [이렇게 하세요] 간염 약 올바른 복용법 안내


    스크립트:
    {text}

    요약:
    """
)

summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

# Combine the contents of all documents for summarization
full_text = '\n'.join([doc.page_content for doc in docs])

# Get a summary of the entire session
session_summary = summary_chain.run(text=full_text)

print(session_summary)


- **Q. 당뇨 약을 일주일 간 끊어도 될까요?**
  - [바꾸세요] 당뇨 약은 중단하지 마세요. 혈관 손상을 막기 위해 지속적으로 복용해야 합니다.
  - [이렇게 하세요] 당화혈색소가 6.5 이하로 떨어지면 약을 줄이는 것을 고려하세요.

- **Q. 스테로이드 사용이 당뇨에 미치는 영향은?**
  - [이렇게 하세요] 스테로이드 사용 시 당뇨 관리하는 의사에게 알리고, 물리치료로 대체하는 것이 좋습니다.

- **Q. 비염과 위염에 아연과 프로폴리스가 효과가 있나요?**
  - [이렇게 하세요] 아연과 프로폴리스는 비염과 위염에 도움이 될 수 있습니다. 함량을 확인하고 복용하세요.

- **Q. 종합 비타민을 먹어도 피로가 해소되지 않아요.**
  - [이렇게 하세요] 비타민 B군이 강화된 제품을 선택하고, 근력 운동을 병행하세요.

- **Q. 눈이 감기고 무거운 느낌이 드는데, 오메가 3가 도움이 되나요?**
  - [이렇게 하세요] 오메가 3는 눈의 기름층 형성에 도움을 주며, 생리 불순에도 효과가 있습니다.

- **Q. 다리가 붓는 느낌이 드는데, 어떻게 해야 하나요?**
  - [이렇게 하세요] 하체 근육량을 늘리기 위해 운동을 하세요. 오메가 3와 프로폴리스 아연 C를 복용하세요.

- **Q. 영양제를 어떻게 복용해야 하나요?**
  - [이렇게 하세요] 지용성 비타민과 오메가 3는 식후에, 수용성 비타민은 식전이나 식후 상관없이 복용하세요.

- **Q. 간염 보균자로서 약을 복용할 때 주의할 점은?**
  - [이렇게 하세요] 약 복용 시간을 일정하게 유지하고, 항산화 영양제를 꾸준히 드세요.

- **Q. 고지혈증 약을 끊어도 되나요?**
  - [이렇게 하세요] 피검사를 통해 수치를 확인하고, 필요 시 약을 복용하세요.

- **Q. 감기약과 타이레놀을 함께 먹어도 되나요?**
  - [이렇게 하세요] 감기 증상에 맞는 약을 선택하고, 타이레놀은 하루 8개 이상 복용하지 마세요.

- **Q. 유산균과 프로바이오틱스를 함께 먹어도

In [41]:
# # Define the prompt template for extracting subtopics
# subtopics_prompt = PromptTemplate(
#     input_variables=["summary"],
#     template="""
#     다음은 상담 세션의 요약입니다. 이 요약에서 논의된 주제들을 10개에서 30개 사이로 추출해 주세요.
#     각 주제에 대해 간단한 설명을 제공하고, 관련된 요약 부분을 포함해 주세요.
#     각 주제는 다음과 같은
#     요약:
#     {summary}

#     주제 목록:
#     """
# )

# # Create the chain for extracting subtopics
# subtopics_chain = LLMChain(llm=llm, prompt=subtopics_prompt)

# # Generate subtopics
# subtopics_output = subtopics_chain.run(summary=session_summary)
subtopics_output= session_summary

In [42]:
# Split the output into a list of subtopics
subtopics_list = subtopics_output.strip().split('\n')

# Clean up the list
subtopics_list = [subtopic.strip() for subtopic in subtopics_list if subtopic.strip() != '']


In [44]:
def ms_to_minutes_seconds_str(milliseconds):
    seconds = milliseconds // 1000  # Total seconds
    minutes = seconds // 60  # Calculate minutes
    remaining_seconds = seconds % 60  # Calculate remaining seconds
    return f"{minutes}분 {remaining_seconds}초:  "  # Format the output string

for subtopic in subtopics_list:
    # Remove numbering if present
    if subtopic[0].isdigit() and subtopic[1] in ['.', ')', ' ']:
        subtopic = subtopic[2:].strip()

    # Retrieve relevant script portions
    relevant_docs = retriever.get_relevant_documents(subtopic)

    # Sort the relevant docs by their start time (from metadata)
    sorted_relevant_docs = sorted(relevant_docs, key=lambda doc: doc.metadata['start_at'])

    # Format the output with time and context in sequential order
    start_at_and_context = "\n\n".join(
        [ms_to_minutes_seconds_str(doc.metadata['start_at']) + doc.page_content for doc in sorted_relevant_docs]
    )

    print(f"**주제:** {subtopic}")
    print(f"**관련 스크립트 부분:**\n{start_at_and_context}")
    print("\n---\n")


**주제:** - **Q. 당뇨 약을 일주일 간 끊어도 될까요?**
**관련 스크립트 부분:**
0분 42초:  다른 글 먹더라도, 그리고 예를 들어서 건강검진 받을 때 한 일주일 전에서부터 아스트림 같은 걸 제가 끊어버리거든요.

1분 2초:  당뇨가요 예.

71분 10초:  약도 잘 안 먹고 하니까 계속 이렇게 가다가 보니까 한 일주일 지나면 괜찮아지니까 감기라는 게 이렇게 해서 하는데 아닌 것 같고.

77분 43초:  아니, 저기 식기마자 두 알씩 해서 한 2주 먹으면 되게 위에 좋다고 얘기해서 그렇게 먹고 있는데 효과 좀 보셨어요.

82분 58초:  이제 고지얼증 약을 저기 줄 수 있을 때는 거기에 인제 의사들이 다 있어서 바로다면 고지얼증 약을 하루에 한 날씩 먹는 걸 받았었는데 이 까먹었는데 그거는 일로 온 다음에 좀 번거로워서.

---

**주제:** - [바꾸세요] 당뇨 약은 중단하지 마세요. 혈관 손상을 막기 위해 지속적으로 복용해야 합니다.
**관련 스크립트 부분:**
0분 29초:  약을 안 먹으면 이 느낌은 없지만 고 기간 동안에 확실히 혈관이 많이 손상을 입거든요.

54분 56초:  수밖에 없는 그러니까 그 노화라는 거는, 그러니까 이제 이 약의 어떤 성분을 몸속에서 이제 수용하고 희석시키기 위해서.

61분 1초:  오히려 이제 아까도 말했지만 노화 촉진을 좀 예방해서 항산화 영양제는 항상 좀 드시는 게 좋다.

84분 31초:  이제 어딘가 혈관이 막히거나 이런 그쵸. 그런 것 때문에 너무 미루지 말고 한번 검사받으시면 좋을 것 같습니다.

87분 57초:  왜냐하면 또 고지혈증 약 오래 드셨으면 고 조혈증 약이 몸에서 대사될 때 이 코인자임 큐텐을 많이 소비해요.

---

**주제:** - [이렇게 하세요] 당화혈색소가 6.5 이하로 떨어지면 약을 줄이는 것을 고려하세요.
**관련 스크립트 부분:**
1분 41초:  쓸려 많이 좋아셨다. 그래서 6, 6.5 이하로 가면은 그때부터 줄일 거를 고려하세요.

61분 1초:  